This notebook is for combining all of the RESULTS.CSV files across all participant folders

Plan: Cycle through all of the folders, obtain name and other rows, add this to a dataframe, then export this as a giant csv

Will also then go in and add if it was TB- or TB+ based on Excel sheet I already have

In [4]:
import os
import pandas as pd

parent_folder = '/Users/tylerstepaniak/Library/Mobile Documents/com~apple~CloudDocs/IGHS/Capstone/Data/GC Files'

# Walk through all subdirectories of the parent folder
for root, dirs, files in os.walk(parent_folder):
    if 'RESULTS.CSV' in files:
        file_path = os.path.join(root, 'RESULTS.CSV')
        try:
            df = pd.read_csv(file_path, skiprows=7, header=None)  # reads from column 8, down.
            subset = df.iloc[:, 1:8].copy()  # grabs columns B to H
            subset.columns = ["PK", "RT", "Area Pct", "Library/ID", "Ref", "CAS", "Qual"]
            subset["PID"] = os.path.basename(root)  # grabs PID
            dfs.append(subset)
        except Exception as e:
            print(f'Error reading {file_path}: {e}')

# adds everything to one DF
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    final_df = final_df.sort_values(by=["PID", "PK"])
    print(final_df)
    final_df.to_csv('intial_output.csv', index=False)  # pushes to csv file
else:
    print("Something went wrong, go grab a coffee")

Empty DataFrame
Columns: [PID, CAS, ID, RT, AreaPct]
Index: []
        PK       RT Area Pct                                      Library/ID  \
21035    1   3.1552   0.0444                             Heptane, 2,3-epoxy-   
21036    2   3.1678   0.0407                                     Heptadecane   
21037    3   3.2332   0.0085         Acetic acid, [(aminocarbonyl)amino]oxo-   
21038    4   3.3008   0.0657                            1-Propene, 2-methyl-   
21039    5   3.3468   0.0058                                         Butanal   
...    ...      ...      ...                                             ...   
53105  196  19.8556   0.0602  Naphthalene, 1,2,3-trimethyl-4-propenyl-, (E)-   
53106  197  19.9303   0.0397  Naphthalene, 1,2,3-trimethyl-4-propenyl-, (E)-   
53107  198  20.4434   0.0272                           Phenol, p-tert-butyl-   
53108  199  20.5981   0.0439             Phenol, 3,5-bis(1,1-dimethylethyl)-   
53109  200  20.8514   0.0362                        Carba

Next step was a QC check comparing the output of:
ls -d */ | sed 's:/$::' | pbcopy
from terminal with the output excel file, which is now shown as 'verified_inital_output.csv' and completed in Excel.

QC check passed.

Now, going to work on organizing the data.

TODO: Will need to rerun once more demographics are obtained.

In [20]:
# generating blank df

df_main1 = pd.DataFrame(
    {'PID': [], 'CAS': [], 'Library/ID': [], 'RT': [], 'Area Pct': []}
    )

# print(df_main1)  # checking

verified_initial_output_df = pd.read_csv('initial_output.csv')
# print(verified_initial_output_df)  # checking that it is the correct csv

df_main1 = pd.concat([df_main1, verified_initial_output_df], ignore_index=True)
df_main1.rename(columns={'Library/ID': 'ID'})
df_main1.drop(['Ref', 'Qual'], axis=1, inplace=True)
# print(df_main1)

# Renaming file names to standardize them and adding demographics
df_demographics = pd.read_excel('/Users/tylerstepaniak/Library/Mobile Documents/com~apple~CloudDocs/IGHS/Capstone/Data/Master Data Spectra.xlsx')
# print(df_demographics)
df_main1 = pd.merge(df_main1, df_demographics, on='PID', how='left')
# print(df_main1)
df_main1.drop(['PID', 'Combined ID', 'PID-temp'], axis=1, inplace=True)
# print(df_main1)
df_main1.rename(columns={'Combined ID Clean': 'PID', 'Library/ID': 'ID'}, inplace=True)

df_main1 = df_main1[['PID', 'File Name Standardized', 'Study', 'TB Status', 'HIV Status', 'Age', 'BMI', 'PK', 'CAS', 'ID', 'RT', 'Area Pct']]
df_main1
df_main1.to_csv('df_main1.csv', index=False)

Data has now been placed into a master dataframe caled df_main1.csv